In [ ]:
!conda create --name DEMSTL
!conda activate DEMSTL


In [1]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2


In [2]:
import os 
import sys
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import osmnx as ox
%matplotlib qt

from buildings import *
from dem2stl import *

In [3]:
sys.path.append('..\\numpy2stl')
import numpy2stl as np2stl
from osm2stl import *

In [4]:
import trimesh
import glob


In [5]:
%matplotlib qt

In [100]:
from shapely.ops import cascaded_union
from shapely.geometry import Polygon,MultiPolygon
from shapely.geometry import polygon

def polygon_to_perimeter(poly):
    
    poly = polygon.orient(poly)
    
    verts,peri = [],[]
    n_v = 0
    exter = np.array(poly.exterior.coords)
    exter = exter[:-1]
    verts.extend(exter)
    peri.append( np.arange(len(exter) + n_v ))
    n_v = len(exter) + n_v 
    
    
    inter = poly.interiors
    for p in inter:
        pts = p.coords[:-1]
        verts.extend( pts )
        peri.append( np.arange(len(pts)) + n_v )
        n_v = len(pts) + n_v             
               
    verts = np.array(verts)
    
    perimeters = []
    for line_idx in peri:
        line = verts[line_idx]
        
        angles = get_perimeter_angles( line) 
        simpified_line = np.array(line_idx[  (angles < 179) | (angles > 181) ])
        perimeters.append(simpified_line)
    

    return verts,perimeters

def polygon_to_prism(polygons,heights,base_val=0):
    all_triangles = []

    for n,poly in enumerate(polygons):
        print(n)
        #if poly.area < 500: continue        
        
        verts, peri = polygon_to_perimeter(poly)
        verts = np.concatenate((verts, verts[:,0:1]*0),axis=1)
        
        verts[:,2] = heights[n]
        try:
            _, faces = np2stl.simplify_surface(verts, peri)
        except: 
            continue
        
        #    print(verts)
        ## Add Z value
        top_tris = verts[faces]
        all_triangles.append( top_tris )
        wall_tris = np2stl.perimeter_to_walls(verts, peri, floor_val=base_val)
        all_triangles.append( wall_tris )

    return all_triangles

def shapely_to_buildings(shp_poly, z0=1,z1=39,polygons=None):

    if polygons is None:    polygons = []
        
    for poly in shp_poly.geoms:
        p = {}
        p['roof_height'] = z1
        p['base_height'] = z0
        p['points'] = np.array(poly.exterior.coords).T
        polygons.append(p)
        
    return polygons

def triangulate_buildings(polygons):

    triangles = []

    for _,p in enumerate( polygons ):

        roof = p['z1'] 
        base = p['z0'] 
        vert = p['points'].T

        #if (np.isclose(vert[0],vert[-1])):   
        vert = vert[:-1]

        zdim = np.zeros((len(vert),1)) + roof
        vert = np.concatenate([vert, zdim],axis=1)
        
        tri = np2stl.polygon_to_prism(vert, base_val=base)
        triangles.append( tri )

    triangles = np.concatenate(triangles)   
    return triangles

def boundry_to_poly(GEO_poly):
    pts = np.array(GEO_poly.exterior.coords).T
    p = {"points":pts,"roof_height":0,"base_height":-30}
    polygons = [p]

    return polygons

def get_waterways( GEO ):
    
    ftpt = ox.footprints_from_polygon(GEO, footprint_type="natural")    
    
    x = ftpt[ftpt["natural"]=="water"]
    x = x.dropna(axis=1, how='all')
    x = x[["geometry","name","waterway","natural"]]
    areas = [i["geometry"].area*10000000 for n,i in x.iterrows()]
    x["areas"] = areas
    x = x[x["areas"]>1]
    
    polys = [ i["geometry"].intersection(GEO) for n,i in x.iterrows()]
    x["geometry"] = polys
    x = ox.project_gdf(x)
    return x 

def map(low_in, high_in, low_out, high_out, qx):

  ix = (qx - low_in)
  ix = (ix / (high_in - low_in))

  ix = ix * (high_out - low_out)
  ix = ix + low_out

  return ix

In [7]:
pd.set_option('display.max_rows', None)

In [10]:

gdf = ox.features_from_place("Granada,Spain", tags)
gdf = ox.features_from_place("Albaicín,Spain", tags)

In [328]:
fn = "C:\\Users\\eac84\\Desktop\\Desktop\\Tasks\\srtm_tifs\\*.jpg"
xl_fn = glob.glob(fn)
im = plt.imread(xl_fn[0])

c:\Users\eac84\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def collect_building_heights(gdf, im, coor_lims):

    H = get_base_height(gdf, im, coor_lims)   
     
    gdf["heights"] = building_heights(gdf)*.05

    gdf["topo_base"] = H
    gdf["z0"] = gdf["topo_base"]
    gdf["z1"] = gdf["z0"]+ gdf["heights"]

    building_poly = get_polygons(gdf)
    tris = triangulate_buildings(building_poly)

    return tris

def get_base_height(gdf, im, coor_lims):

    x_ = gdf["geometry"]

    ###########################
    xy_list = []
    for x in x_:
      xy_list.append([x.centroid.x,  x.centroid.y])
    xy_list = np.array(xy_list)

    ###########################
    im_lims = ((0,im.shape[0]),(0,im.shape[1]))
    Nc, Wc = coor2im(coor_lims, im_lims, xy_list)
    Nc,Wc = Nc.astype(int), Wc.astype(int)
    #################
    Nx,Sx = Nc.max(),Nc.min()
    Ex,Wx = Wc.max(),Wc.min()
    ############################
    H = im[Nc, Wc] *.8
    data =  im[int(Sx):int(Nx), int(Wx):int(Ex)] *.8
    #####################
    H = (H) - data.min() + 1

    return H

def coor2im(coor_lims, im_lims, xy_list):

    N0,N1 = coor_lims[0]
    W0,W1 = coor_lims[1]
    X0,X1 = im_lims[0]
    Y0,Y1 = im_lims[1]

    Ncoor = map( N0,N1, X0*1., X1*1., xy_list[:,1])
    Wcoor = map( W0,W1, Y0*1., Y1*1., xy_list[:,0])

    return Ncoor, Wcoor


In [355]:
######################
Nx,Sx,Wx,Ex = bounds

data =  im[int(Sx):int(Nx), int(Wx):int(Ex)] *.8
data = data - data.min() + 1

####################
facet = np2stl.numpy2stl(data)
solid = np2stl.Solid(facet)
vx = solid.vertices.copy().astype(float)
########################
vx = reposition_dem(vx, im_lims, coor_lims, bounds)
########################
vx[:,2] = vx[:,2] * scale
vx = vx*1000
#########################


c:\Users\eac84\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Creating top...
Creating walls...
Creating bottom...


In [ ]:
def reposition_dem(vx, im_lims, coor_lims, bounds):

  Nx,Sx,Wx,Ex = bounds

  x,y = vx[:,1],vx[:,0]
  x = x + Sx
  y = y + Wx
  imcoor = np.array((y,x)).T*1.
  y, x = coor2im(im_lims, coor_lims, imcoor)
  vx[:,0], vx[:,1] = y, x

  return vx 

In [354]:
tags = {'building': True}
center = (37.17827, -3.5925189) 
gdf = ox.features.features_from_point(center, tags, dist=1000)

coor_lims = ((38, 37),(-4, -3))

tris = collect_building_heights(gdf, im, coor_lims)
vertices, faces = np2stl.vertices_to_index(tris)

##################
vertices[:,[1,0]] = vertices[:,[0,1]]
vertices[:,2] = vertices[:,2] *scale
vertices = vertices*1000

#building_poly = MultiPolygon(building_poly)
#building_poly = unary_union(building_poly)
#building_list = shapely_to_buildings(building_poly , z0=0,z1=10)

c:\Users\eac84\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Adding Buildings
Making Polygons


In [344]:
print(vertices.mean(axis=0), vx.mean(axis=0))
print(vertices.max(axis=0) - vertices.min(axis=0), vx.max(axis=0)- vx.min(axis=0))

c:\Users\eac84\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[ 3.71768097e+04 -3.59727079e+03  1.10189902e+00] [ 3.71784569e+04 -3.59283105e+03  1.16870074e+00]
[22.7281     28.1055      3.19355735] [21.66064982 27.21466259  3.74895862]


c:\Users\eac84\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


opening - Napari


In [356]:
import napari
print("opening - Napari")
v = napari.current_viewer()
if v is None: v = napari.Viewer()
v.layers.clear()


surface1 = (vertices,faces)
s = v.add_surface(surface1)
s.wireframe.visible = True
###
surface = (vx, solid.faces)
s = v.add_surface(surface)
s.wireframe.visible = True
#s.normals.face.visible = True
#s.normals.vertex.visible = True

opening - Napari


In [ ]:

mesh = trimesh.Trimesh(vertices,faces)

In [ ]:
mesh.export("Granada_buildings.stl")

c:\Users\eac84\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(array([ 3.71768097e+04, -3.59727079e+03, -3.75786216e+00]),
 array([ 3.75681566e+04, -3.15362209e+03,  2.07166104e+00]),
 array([22.7281    , 28.1055    ,  3.19355735]),
 array([21.66064982, 27.21466259,  7.22021661]))

In [ ]:
%matplotlib qt

In [201]:
from skimage import filters,transform
 
outshape = np.array(data.shape)*2
filt = transform.resize(data, outshape)
filt = filters.gaussian( filt , sigma=3, truncate=3)
filt = filters.median(filt, np.ones((3,3)))

plt.figure()
plt.imshow(filt)

#data = dem.data
facet = np2stl.numpy2stl(filt)
solid = np2stl.Solid(facet)
#solid.simplify()
fn = "granada_topo.stl" 
solid.save_stl(fn)

Creating top...
Creating walls...
Creating bottom...


..\numpy2stl\numpy2stl\save.py:39: ResourceWarning: unclosed file <_io.BufferedWriter name='granada_topo.stl'>
  f = open(file_name, 'wb')


KeyboardInterrupt: 

In [ ]:
x = 660
y = 1470
plt.imshow(im[-780:-570,1370:1570])

In [ ]:
r0 = 2820
r1 = 3030
c0 = 1370
c1 = 1570

mx = im[r0:r1, c0:c1]
plt.imshow(mx)

In [ ]:
plt.imshow(im)
plt.plot(Wcoor,Ncoor, "ro")

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.plot(Ncoor,Wcoor,H, "o")